In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np

MAX_PAGE = 1
BRANDS = ['Dell', 'Acer']


def soupify(BRAND, PAGE_NUMBER):
    url = 'http://www.cdiscount.com/search/10/' + BRAND + '/.html?TechnicalForm.SiteMapNodeId=0&TechnicalForm.DepartmentId=10&TechnicalForm.ProductId=&hdnPageType=Search&TechnicalForm.SellerId=&TechnicalForm.PageType=SEARCH_AJAX&TechnicalForm.LazyLoading.ProductSheets=False&NavigationForm.CurrentSelectedNavigationPath=0&FacetForm.SelectedFacets.Index=0&FacetForm.SelectedFacets.Index=1&FacetForm.SelectedFacets.Index=2&FacetForm.SelectedFacets.Index=3&FacetForm.SelectedFacets.Index=4&FacetForm.SelectedFacets.Index=5&FacetForm.SelectedFacets.Index=6&FacetForm.SelectedFacets.Index=7&FacetForm.SelectedFacets.Index=8&FacetForm.SelectedFacets.Index=9&GeolocForm.ConfirmedGeolocAddress=&SortForm.SelectedNavigationPath=&ProductListTechnicalForm.Keyword=dell&page=' + str(PAGE_NUMBER) + '&_his_'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser') 
    return soup        


def extractPricesFromPage(BRAND, PAGE_NUMBER):
    
    soup = soupify(BRAND, PAGE_NUMBER)
    START_PRICES = np.array([])
    END_PRICES = np.array([])
    
    # Extract  start and end prices 
    
    finderAllPrices = soup.find_all(class_='prdtBZPrice')
    finderStartPrices = soup.find_all(class_="prdtPrSt")
    finderEndPrices = soup.find_all(class_="price")
    
    k = 0   
    for i in range(0, len(finderAllPrices) - 1):
        
        startPriceToAdd = float(finderStartPrices[k].text.split(',')[0]) + 0.01 * float(finderStartPrices[k].text.split(',')[1])
        discountedPriceToAdd = float(finderEndPrices[i].text.split('€')[0]) + 0.01 * float(finderEndPrices[i].text.split('€')[1])   
              
        if 'prdtPrSt' in str(finderAllPrices[i]):
                       
            START_PRICES = np.append(START_PRICES, startPriceToAdd)
            END_PRICES = np.append(END_PRICES, discountedPriceToAdd)

            k += 1    
            
        else:
            
            START_PRICES = np.append(START_PRICES, discountedPriceToAdd)
            END_PRICES = np.append(END_PRICES, discountedPriceToAdd)
    
    # Test if both arrays are same length
    
    if len(START_PRICES) == len(END_PRICES):        
        return START_PRICES, END_PRICES
    else:
        return "List start and end prices not same length"

    
def discountRates(BRAND, MAX_PAGE): 
    
    ALL_START_PRICES = np.array([])
    ALL_END_PRICES = np.array([])

    for PAGE_NUMBER in range(1, MAX_PAGE + 1):
        NEW_START_PRICES, NEW_END_PRICES = extractPricesFromPage(BRAND, PAGE_NUMBER)
        ALL_START_PRICES = np.append(ALL_START_PRICES, NEW_START_PRICES)
        ALL_END_PRICES = np.append(ALL_END_PRICES, NEW_END_PRICES)
      
    return ((ALL_START_PRICES - ALL_END_PRICES) * 100 / ALL_START_PRICES).mean()

In [ ]:
"""
Who from Dell or Acer have the best discount rate on Cdiscount.com ?
"""

bestDiscountRate = 0
bestBrand = ''

for brand in BRANDS:
    if discountRates(brand, MAX_PAGE) > bestDiscountRate:
        bestDiscountRate = discountRates(brand, MAX_PAGE)
        bestDiscounter = brand    
    print("discountRate_" + brand, discountRates(brand, MAX_PAGE).round(2), "%")
    
print("Best dicounter is:", brand)    